In [17]:
# Instalacja PyTorch Lightning
# !pip install pytorch-lightning

In [18]:
# 1. System i Dane
import os
import pandas as pd
import numpy as np

# 2. PyTorch (Silnik)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 3. PyTorch Lightning (Trener)
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# 4. Preprocessing
# To jedyny element z zewnątrz, który zostawiamy, bo ręczne pisanie
# normalizacji danych (skalowania do 0-1) to strata czasu i ryzyko błędu.
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit # ### NOWE: Potrzebne do podziału silników


# Ustawienie ziarna losowości dla powtarzalności
pl.seed_everything(42)

Seed set to 42


42

In [19]:
!git clone https://github.com/edwardzjl/CMAPSSData.git

fatal: destination path 'CMAPSSData' already exists and is not an empty directory.


In [22]:
class CMAPSS_Preprocessor:
    def __init__(self, data_path, sequence_length=30, alpha=0.25, max_rul=125):
        self.data_path = data_path #ściezka do pliku
        self.sequence_length = sequence_length #długość okna czasowego
        self.alpha = alpha  # Współczynnik wygładzania z artykułu
        self.max_rul = max_rul # Przycinanie RUL (Piecewise Linear)

        # Definicja kolumn tablicy na dane
        self.index_cols = ['unit_nr', 'time_cycles']
        self.setting_cols = ['os_1', 'os_2', 'os_3']
        
        # ### ZMIANA START: Definicja wszystkich sensorów oraz tych wybranych w artykule ###
        self.all_sensor_cols = ['s' + str(i) for i in range(1, 22)] # Wszystkie 21 sensorów (do wczytania i wygładzenia)
        
        # Zgodnie z artykułem dla FD001 wybrano tylko te sensory (CSC > 0.75):
        # S2, S3, S4, S7, S8, S11, S12, S13, S15, S17, S20, S21
        self.selected_sensors = ['s2', 's3', 's4', 's7', 's8', 's11', 
                                 's12', 's13', 's15', 's17', 's20', 's21']
        
        # Do wczytania używamy wszystkich nazw
        self.cols = self.index_cols + self.setting_cols + self.all_sensor_cols
        # ### ZMIANA KONIEC ###

        # Skaler (będziemy go uczyć tylko na treningu)(?) - normalizacja odczytów z czujników do zakresu 0-1
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def process(self, file_name='FD001'):
        # 1. Wczytanie danych z  plików do dataframes (tabeli)
        train_df = pd.read_csv(f'{self.data_path}/train_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_df = pd.read_csv(f'{self.data_path}/test_{file_name}.txt', sep=r'\s+', header=None, names=self.cols)
        test_rul_df = pd.read_csv(f'{self.data_path}/RUL_{file_name}.txt', sep=r'\s+', header=None, names=['RUL'])

        # 2. Obliczenie etykiet RUL dla treningu
        train_df = self._add_rul(train_df, is_test=False)

        # 3. Wygładzanie danych (Exponential Smoothing) - WAŻNE!
        # ### KOMENTARZ: Wygładzamy wszystkie sensory, zanim odrzucimy te niepotrzebne
        train_df = self._smooth_data(train_df)
        test_df = self._smooth_data(test_df)

        # ### ZMIANA START: Selekcja cech i Normalizacja ###
        # Bierzemy TYLKO wybrane sensory (bez ustawień os_*, bo artykuł ich nie używa dla FD001)
        feats = self.selected_sensors 
        
        # 4. Normalizacja (Fit na CAŁYM zbiorze treningowym, Transform na obu)
        # .fit znajduje max i min w zbiorach feats
        self.scaler.fit(train_df[feats])      
        train_df[feats] = self.scaler.transform(train_df[feats])  # normalizacja dataframeu treningowego
        test_df[feats] = self.scaler.transform(test_df[feats])    # normalizacja dataframeu testowego
        # ### ZMIANA KONIEC ###

        # ### NOWE START: Podział zbioru treningowego na Train (90) i Val (10) ###
        # Artykuł mówi: "90 engines are randomly chosen for training and 10 for validation"
        splitter = GroupShuffleSplit(n_splits=1, train_size=0.9, random_state=42)
        train_idx, val_idx = next(splitter.split(train_df, groups=train_df['unit_nr']))
        
        real_train_df = train_df.iloc[train_idx].copy() # 90 silników
        real_val_df = train_df.iloc[val_idx].copy()     # 10 silników
        # ### NOWE KONIEC ###

        # 5. Generowanie okien czasowych (Sliding Window)
        # ### ZMIANA: Generujemy osobno dla Train i Val ###
        X_train, y_train = self._gen_sequence(real_train_df, feats)
        X_val, y_val = self._gen_sequence(real_val_df, feats)
        
        # Uwaga: Dla testu w C-MAPSS zazwyczaj bierze się tylko OSTATNIE okno,
        # bo plik RUL_FD001.txt zawiera tylko jedną liczbę dla każdego silnika (RUL na samym końcu).
        X_test, y_test = self._gen_test_sequence(test_df, test_rul_df, feats)

        # ### ZMIANA: Zwracamy teraz 6 tablic zamiast 4 ###
        return X_train, y_train, X_val, y_val, X_test, y_test

    def _add_rul(self, df, is_test=False):
        # Grupowanie po silniku, znalezienie max cyklu
        max_life = df.groupby('unit_nr')['time_cycles'].transform('max')
        df['RUL'] = max_life - df['time_cycles']

        # Implementacja Piecewise Linear RUL (ucinamy powyżej 125)
        df['RUL'] = df['RUL'].clip(upper=self.max_rul)
        return df

    def _smooth_data(self, df):
        # WAŻNE: Musimy grupować po 'unit_nr'!
        # Nie możemy pozwolić, by wygładzanie przeniosło się z końca Silnika 1 na początek Silnika 2.
        
        # ### ZMIANA: Używamy self.all_sensor_cols zamiast self.sensor_cols ###
        df[self.all_sensor_cols] = df.groupby('unit_nr')[self.all_sensor_cols].transform(     #wg Gemini .ewm.mean zrobi to samo co wzór na wygładzanie z artykułu
            lambda x: x.ewm(alpha=self.alpha, adjust=False).mean()
        )
        return df

    def _gen_sequence(self, df, feature_cols):
        X, y = [], []
        data_array = df[feature_cols].values    # data_array: Czysta macierz liczb [Liczba cykli, 12 sensorów]
        target_array = df['RUL'].values     # target_array: Wektor RUL
        unit_ids = df['unit_nr'].values

        # Iterujemy, ale musimy uważać, żeby okno nie "przeskoczyło" między silnikami
        for i in range(len(df) - self.sequence_length):
            # Sprawdzamy czy okno mieści się w JEDNYM silniku
            if unit_ids[i] == unit_ids[i + self.sequence_length]: #id silnika takie same na początku i końcu sekwencji
                # Dodajemy okno
                X.append(data_array[i : i + self.sequence_length]) 
                # Dodajemy cel (RUL w ostatnim kroku tego okna)
                y.append(target_array[i + self.sequence_length - 1])

        # np.array(X) ma wymiar: [N, T, 12]
        return np.array(X), np.array(y)

    def _gen_test_sequence(self, test_df, truth_df, feature_cols):
        X, y = [], []
        # Dla zbioru testowego bierzemy tylko OSTATNIE cykle każdego silnika
        # I przypisujemy mu prawdziwy RUL z pliku RUL_FD001.txt

        true_ruls = truth_df['RUL'].values   #truth_df Wymiar: [100 wierszy, 1 kolumna]

        for unit_id in test_df['unit_nr'].unique():     #.unique usuwa wielokrotne id
            # Wyciągamy dane jednego silnika
            temp_df = test_df[test_df['unit_nr'] == unit_id]

            if len(temp_df) >= self.sequence_length:
                # Bierzemy ostatnie 'sequence_length' cykli
                window = temp_df[feature_cols].values[-self.sequence_length:]
                X.append(window)
                # Bierzemy prawdziwy RUL (indeks unit_id-1, bo silniki są od 1, tablica od 0)
                y.append(true_ruls[unit_id - 1])

        return np.array(X), np.array(y)

In [23]:
class CMAPSSDataModule(pl.LightningDataModule):
    def __init__(self, data_path='CMAPSSData', batch_size=10, sequence_length=30):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size # Wg artykułu batch=10
        self.sequence_length = sequence_length
        self.preprocessor = CMAPSS_Preprocessor(data_path, sequence_length)

    def setup(self, stage=None):
        # Tu dzieje się cała magia przygotowania danych
        # ### ZMIANA: Odbieramy 6 zmiennych (Train, Val, Test) ###
        X_train_np, y_train_np, X_val_np, y_val_np, X_test_np, y_test_np = self.preprocessor.process('FD001')

        # Konwersja Numpy -> PyTorch Tensor
        self.train_X = torch.tensor(X_train_np, dtype=torch.float32)
        self.train_y = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1) # [N] -> [N, 1]

        # ### NOWE: Tensory walidacyjne ###
        self.val_X = torch.tensor(X_val_np, dtype=torch.float32)
        self.val_y = torch.tensor(y_val_np, dtype=torch.float32).unsqueeze(1)

        self.test_X = torch.tensor(X_test_np, dtype=torch.float32)
        self.test_y = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)

        print(f"Dane przygotowane!")
        print(f"Trening (90 silników): {self.train_X.shape}") 
        print(f"Walidacja (10 silników): {self.val_X.shape}") 
        print(f"Test (100 silników, ostatnie okna): {self.test_X.shape}") 

    def train_dataloader(self):
        # Shuffle=True jest kluczowe dla treningu!
        dataset = torch.utils.data.TensorDataset(self.train_X, self.train_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # ### ZMIANA: Używamy teraz prawdziwego zbioru walidacyjnego (10 wydzielonych silników) ###
        # Używamy shuffle=False dla walidacji
        dataset = torch.utils.data.TensorDataset(self.val_X, self.val_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
    
    # ### NOWE: Dodajemy loader dla zbioru testowego (do sprawdzenia ostatecznych wyników po treningu) ###
    def test_dataloader(self):
        dataset = torch.utils.data.TensorDataset(self.test_X, self.test_y)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

In [24]:
# 1. Inicjalizacja DataModule
dm = CMAPSSDataModule(batch_size=10, sequence_length=30)

# 2. Uruchomienie setup (wczytanie i przetworzenie)
dm.setup()

# 3. Sprawdzenie jednej paczki danych
# Pobieramy jeden batch z loadera
x_batch, y_batch = next(iter(dm.train_dataloader()))

print("\n--- Sprawdzenie Batcha ---")
print(f"Wymiary wejścia (X): {x_batch.shape}")
# Powinno być: torch.Size([10, 30, 24]) -> 10 próbek, 30 cykli, 24 cechy
print(f"Wymiary etykiet (y): {y_batch.shape}")
# Powinno być: torch.Size([10, 1]) -> 10 wyników RUL

print("\nPrzykładowy RUL z batcha:", y_batch[0].item())

Dane przygotowane!
Trening (90 silników): torch.Size([15757, 30, 12])
Walidacja (10 silników): torch.Size([1874, 30, 12])
Test (100 silników, ostatnie okna): torch.Size([100, 30, 12])

--- Sprawdzenie Batcha ---
Wymiary wejścia (X): torch.Size([10, 30, 12])
Wymiary etykiet (y): torch.Size([10, 1])

Przykładowy RUL z batcha: 104.0


In [28]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

class DLSTMRegressor(pl.LightningModule):
    # ### ZMIANA: Domyślny input_size ustawiony na 12 (zgodnie z nowym pre-processingiem)
    def __init__(self, input_size=12, hidden_size=100, num_layers=5, dropout=0.7, learning_rate=0.001):
        super().__init__()
        self.save_hyperparameters() 

        # Definicja architektury DLSTM zgodnie z artykułem
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout, 
            batch_first=True
        )

        # Warstwa wyjściowa
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] # Ostatni krok czasowy
        rul_prediction = self.fc(out)
        return rul_prediction

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        
        # Logowanie metryk (Czysty pasek: on_step=False)
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        
        loss = self.criterion(y_hat, y)
        rmse = torch.sqrt(loss)
        
        # Obliczenie metryki Score
        h = y_hat - y
        score1 = torch.exp(-h[h < 0] / 13) - 1
        score2 = torch.exp(h[h >= 0] / 10) - 1
        score = torch.sum(score1) + torch.sum(score2)
        
        # ### ZMIANA: Dodano on_step=False, on_epoch=True dla czystości logów
        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_rmse', rmse, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_score', score, prog_bar=True, on_step=False, on_epoch=True)
        
        return loss
    
    # ### NOWE: Dodano test_step, aby policzyć ostateczny wynik na zbiorze testowym (Tabela 3 w artykule)
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        rmse = torch.sqrt(loss)
        h = y_hat - y
        score = torch.sum(torch.exp(-h[h < 0] / 13) - 1) + torch.sum(torch.exp(h[h >= 0] / 10) - 1)
        
        self.log('test_rmse', rmse, on_epoch=True)
        self.log('test_score', score, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)


# --- KONFIGURACJA TRENINGU ---

# 1. Dane
dm = CMAPSSDataModule(batch_size=10, sequence_length=50) 

# 2. Model
# ### ZMIANA: input_size=12 (Bo wybraliśmy 12 sensorów z artykułu)
model = DLSTMRegressor(
    input_size=12,   
    hidden_size=100, 
    num_layers=5,    
    dropout=0.7,     
    learning_rate=0.001 
)

# 3. Callbacki (Checkpoint + EarlyStopping)
checkpoint_callback = ModelCheckpoint(
    monitor='val_rmse', 
    mode='min', 
    save_top_k=1, 
    filename='best-dlstm-{epoch}-{val_rmse:.2f}'
)

# ### ZALECANE: Dodaj EarlyStopping, żeby nie czekać 1000 epok, jeśli model przestanie się uczyć
early_stop_callback = EarlyStopping(
    monitor="val_rmse", 
    min_delta=0.001, 
    patience=4, # Jeśli przez 20 epok wynik się nie poprawi, stop
    verbose=True, 
    mode="min"
)

# 4. Trener
trainer = pl.Trainer(
    max_epochs=15,               # ### ZMIANA: 1000 epok zgodnie z Tabelą 2 w artykule
    accelerator="auto",       
    callbacks=[checkpoint_callback, early_stop_callback], # Dodano early stopping
    log_every_n_steps=10
)

# 5. Start
trainer.fit(model, dm)

# 6. Testowanie (To da wynik porównywalny z Tabelą 3 artykułu)
print("Uruchamianie testu na najlepszym modelu...")
trainer.test(model, ckpt_path="best", datamodule=dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])



  | Name      | Type    | Params | Mode  | FLOPs
------------------------------------------------------
0 | lstm      | LSTM    | 368 K  | train | 0    
1 | fc        | Linear  | 101    | train | 0    
2 | criterion | MSELoss | 0      | train | 0    
------------------------------------------------------
368 K     Trainable params
0         Non-trainable params
368 K     Total params
1.476     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])


c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_rmse            40.68370819091797
       test_score            1364.87158203125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_rmse': 40.68370819091797, 'test_score': 1364.87158203125}]

In [ ]:
# # import torch
# # import torch.nn as nn
# # import torch.nn.functional as F
# # import pytorch_lightning as pl

# class DLSTMRegressor(pl.LightningModule):
#     def __init__(self, input_size=24, hidden_size=100, num_layers=5, dropout=0.7, learning_rate=0.001):
#         super().__init__()
#         self.save_hyperparameters() # Zapisuje parametry do self.hparams

#         # Definicja architektury DLSTM zgodnie z artykułem
#         # input_size=24 (3 ustawienia + 21 czujników)
#         # batch_first=True, bo nasze dane mają kształt (Batch, Seq, Features)
#         self.lstm = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             dropout=dropout, 
#             batch_first=True
#         )

#         # Warstwa wyjściowa (Fully Connected) - mapuje wyjście LSTM na pojedynczą wartość RUL
#         self.fc = nn.Linear(hidden_size, 1)

#         # Funkcja straty (MSE)
#         self.criterion = nn.MSELoss()

#     def forward(self, x):
#         # x shape: (Batch, Sequence_Length, Features)
        
#         # Przejście przez warstwy LSTM
#         # out shape: (Batch, Sequence_Length, Hidden_Size)
#         # _ (hidden states) - ignorujemy
#         out, _ = self.lstm(x)
        
#         # Bierzemy wyjście z OSTATNIEGO kroku czasowego dla każdej próbki w batchu
#         # Many-to-One architecture
#         out = out[:, -1, :] 
        
#         # Przejście przez warstwę liniową
#         rul_prediction = self.fc(out)
#         return rul_prediction

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = self.criterion(y_hat, y)
        
#         # Logowanie metryk
#         self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
        
#         # 1. Obliczenie RMSE (pierwiastek z MSE)
#         loss = self.criterion(y_hat, y)
#         rmse = torch.sqrt(loss)
        
#         # 2. Obliczenie metryki Score (wg wzoru z artykułu/C-MAPSS)
#         # h = (Predicted - Real)
#         h = y_hat - y
        
#         # Jeśli h < 0 (wczesna predykcja): exp(-h/13) - 1
#         # Jeśli h >= 0 (późna predykcja - groźna): exp(h/10) - 1
#         score1 = torch.exp(-h[h < 0] / 13) - 1
#         score2 = torch.exp(h[h >= 0] / 10) - 1
#         score = torch.sum(score1) + torch.sum(score2)
        
#         self.log('val_loss', loss, prog_bar=True)
#         self.log('val_rmse', rmse, prog_bar=True)
#         self.log('val_score', score, prog_bar=True)
        
#         return loss

#     def configure_optimizers(self):
#         # Optymalizator Adam (zgodnie z artykułem)
#         return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [ ]:
# # 1. Konfiguracja danych
# # Zmieniamy sequence_length na 50, zgodnie z Twoją analizą literatury [11]
# # Domyślnie w klasie było 30, ale DataModule jest elastyczny.
# dm = CMAPSSDataModule(batch_size=10, sequence_length=50) 

# # 2. Inicjalizacja modelu DLSTM
# # Parametry zgodne z "najlepszą konfiguracją" z artykułu (Tabela 1 i sekcja 3.4)
# model = DLSTMRegressor(
#     input_size=12,   # Liczba cech (z Twojego pre-processora)
#     hidden_size=100, # 
#     num_layers=5,    # 
#     dropout=0.7,     # 
#     learning_rate=0.001 # Typowa wartość dla Adama, artykuł sugeruje 0.001 w Tabeli 2 
# )

# # 3. Setup Trenera (PyTorch Lightning)
# # Ustawiamy max_epochs na np. 100 (artykuł używał 1000, ale na początek wystarczy mniej)
# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     monitor='val_rmse', 
#     mode='min', 
#     save_top_k=1, 
#     filename='best-dlstm-{epoch}-{val_rmse:.2f}'
# )

# trainer = pl.Trainer(
#     max_epochs=500,             # Dostosuj wg potrzeb
#     accelerator="auto",        # Użyje GPU jeśli dostępne
#     callbacks=[checkpoint_callback],
#     log_every_n_steps=10
# )

# # 4. Start Treningu
# trainer.fit(model, dm)

# # 5. Wyniki walidacji po treningu
# print("Najlepszy wynik walidacji (RMSE):", checkpoint_callback.best_model_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Dane przygotowane!
Trening (90 silników): torch.Size([13957, 50, 12])
Walidacja (10 silników): torch.Size([1674, 50, 12])
Test (100 silników, ostatnie okna): torch.Size([93, 50, 12])



  | Name      | Type    | Params | Mode  | FLOPs
------------------------------------------------------
0 | lstm      | LSTM    | 368 K  | train | 0    
1 | fc        | Linear  | 101    | train | 0    
2 | criterion | MSELoss | 0      | train | 0    
------------------------------------------------------
368 K     Trainable params
0         Non-trainable params
368 K     Total params
1.476     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\marci\miniconda3\envs\gsn_laby\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
